In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pysqlite3
import time
import pandas as pd
import re

conn = pysqlite3.connect('kayzen.db')

In [ ]:
query = """
SELECT
DISTINCT imp.[App Id]
FROM imp
WHERE imp.[App Id] IS NOT NULL
;
"""
pub = pd.read_sql_query(query,conn)

In [ ]:
pubs = pub['App Id'].tolist()
ios = re.compile(r'^[0-9]')
all_ios = list(filter(ios.search, pubs))
andriod = re.compile(r'^com.')
all_android = list(filter(andriod.search, pubs))

In [ ]:
# andoid
android_url = 'https://play.google.com/store/apps/details?id='
cat = []
for app in all_android:
    page_url = android_url + app
    print(page_url)
    page = requests.get(page_url)
    soup = bs(page.text, 'html.parser')
    try:
        category = soup.find(itemprop="genre").text
    except:
        category = None
    cat += [[app, category]]

In [ ]:
# apple
ios_url = 'https://apps.apple.com/us/app/id'
cat_ios = []
for app in all_ios:
    app = app.split('.')[0]
    page_url = ios_url + app
    print(page_url)
    page = requests.get(page_url)
    soup = bs(page.text, 'html.parser')
    try:
        category = soup.find_all(class_="information-list__item__definition")[2].a.text
    except:
        category = None
    cat_ios += [[app, category]]

In [ ]:
# Creating df
all_categories = cat_ios + cat
df = pd.DataFrame(data = all_categories)
df.columns = ['App Id','Category']

In [ ]:
# dropping null values
df.dropna(inplace=True)
# removing spaces on both end
df.Category = df.Category.str.strip()
# export to csv
df.to_csv('Categories.csv', index=False)
# df > sql [Table name: Categories]
df.to_sql('Categories', conn)